<a href="https://colab.research.google.com/github/jiin124/MachineLearning/blob/main/crossvalidation%2Cgrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
wine=pd.read_csv('http://bit.ly/wine-date')

In [3]:
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

In [5]:
sub_input,val_input,sub_target,val_target=train_test_split(train_input,train_target,test_size=0.2,random_state=42)

train_input과 train_target을 다시 train_test_split() 함수에 넣어 훈련세트 sub_input,sub_target 과 검증 세트 val_input val_target을 만든다.여기서 test_size=0.2로 지정해 train_input의 약 20%dmf val_input으로 만들어준다.

함수를 두번 적용해서 훈련세트와 검증 세트로 나눠준 것이다. 훈련세트와 검증세트의 크기를 확인해보자

In [6]:
print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


원래 5197개였던 훈련세트가 4157개로 줄고 검증 세트는 1040이 되었다. 

In [7]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [8]:
from sklearn.model_selection import cross_validate
scores=cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.00785041, 0.0072453 , 0.00764346, 0.00759959, 0.00692868]), 'score_time': array([0.00077581, 0.00063062, 0.00063515, 0.00056696, 0.00055909]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다. 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다. 각 키마다 5개의 숫자가 담겨있다. cross_vaildate() 함수는 기본적으로 5-폴드 교차검증을 수행한다. cv 매개변수에서 폴드 수를 바꿀 수 있다.

In [9]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


교차검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있다. 
한 가지 주의할 점은 cross_vaildate() 는 훈련세트를 섞어 폴드를 나누지 않는다. 앞서 우리는 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없다. 하지만 만약 교차 검증을 할떄 훈련세트를 섞으려면 분할기(spliter)를 지정해야 한다.

In [11]:
from sklearn.model_selection import StratifiedKFold
scores=cross_validate(dt,train_input,train_target,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [14]:
splitter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores=cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))
#n_splits 매개변수는 몇 폴드 교차 검증을 할지 정할 수 있다.

0.8574181117533719


이제 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아보자! 테스트 세트를 사용하지 않고 교차 검증을 통해 좋은 모델을 고르면 된다. 

## 하이퍼파라미터

모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 한다. 사이킷런과 같은 머신러닝 라이브러리를 사용할 때 이런 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현한다. 

사이킷런의 GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다. 별도로 cross_calidate()함수를 호출할 필요가 없다. min_impurity_decease 매개변수의 최적값을 찾아보자

In [15]:
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [16]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [17]:
gs.fit(train_input,train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [19]:
dt=gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


In [20]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [21]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [23]:
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [24]:
params={'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
        'max_depth':range(5,20,1),
        'min_samples_split':range(2,100,10)}

9*15*10=1350 개의 교차검증횟수
기본 5-폴드 교차 검증을 수행하므로 만들어지는 모델의 수는 6750개이다. 
n-job 매개변수를 -1로 정하고 그리드 서치를 실행해보겠다

In [27]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [28]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [29]:
#최상의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


## 랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다 또 너무 많은 매개 변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다. 이럴 떄는 랜덤 서치를 사용한다. 랜덤 서치에는 매개 변수 값의 목록을 전달하는 것이 아니라 매개 변수를 샘플링 팔 수 있는 확률 분포 객체를 전달한다.

In [34]:
from scipy.stats import uniform,randint
#싸이파이의 stats 서브 패키지에 있는 uniform과 randint 클래스는 주어진 범위에서 고르게 값을 뽑는다. 이를 균등 분포에서 샘플링한다고 말한다. 
#randint는 정숫값을 뽑고 uniform은 실숫 값을 뽑는다. 

In [35]:
rgen=randint(0,10)
rgen.rvs(10)

array([2, 7, 5, 7, 2, 4, 0, 0, 6, 8])

In [36]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 95, 104,  92, 124,  93, 108,  82,  96,  99, 107]))

In [37]:
ugen=uniform(0,1)
ugen.rvs(10)

array([0.46924925, 0.01688851, 0.39051703, 0.71536477, 0.11646714,
       0.27444022, 0.61357102, 0.2310073 , 0.96639331, 0.8488185 ])

랜덤서치에 randint와 uniform 클래스 객체를 넘겨주고 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 명령할 수 있다. 샘플링 횟수는 시스템 자원이 허락하는 범위내에서 최대한 크게하는 것이 좋다. 

In [38]:
params={'min_impurity_decrease':uniform(0.0001,0.001),
        'max_depth':randint(20,50),
        'min_samples_split':randint(2,25),
        'min_samples_leaf':randint(1,25),}

In [42]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [43]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [44]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [45]:
dt=gs.best_estimator_
print(dt.score(test_input,test_target))

0.86


# 최적의 모델을 위한 파이퍼파라미터 탐색 마무리..

테스트 세트는 최종 모델을 선택할 때 까지 사용하지 말아햐 한다. 테스트 세트를 사용하지 않고 모델을 평가하려면 또다른 세트가 필요하다. 이를 검증 세트라고 부른다. 혹은 개발 세트라고 한다 검증 세트는 훈련세트 중 일부를 다시 덜어 내어 만든다. 

컴증 세트가 크지 않다면 어떻게 데이터를 나누었는지에 따라 검증 점수가 들쭉날쭉 할 것이다. 훈련한 모델의 성능을 안정적으로 평가하기 위해 검증 세트를 한 번 나누어 모델을 평가하는 것에 그치지 않고 여러번 반복 할 수 있다. 이를 교차 검증이라고 한다. 